----------------------------------------------------
Modelagem de tópicos de documentos provenientes da base de dados do CPDOC.
----------------------------------------------------
Base de dados: Antonio Azeredo da Silveira.
-----------------------------
Número de documentos: 10.428
-----------------------------
Número de imagens: 66.938
-----------------------------

### Topic: Clustering documents

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyLDAvis
import pyLDAvis.gensim

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead

Specifying the path to the files  

In [2]:
#datapath = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected' # path marcelo
datapath = "../textfiles-corrected"
outputs = "../outputs/"
import os

onlyfiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath,f))]

In [3]:
onlyfiles[0:4]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/rsouza/pytho

['txt_0007168_484.txt',
 'txt_0062976_89.txt',
 'txt_0023552_242.txt',
 'txt_0002560_270.txt']

#### make list of txt contents of files

In [4]:
count = 0
percentil = int(len(onlyfiles)/100)
textos = []

for txt in onlyfiles:
    with open(os.path.join(datapath,txt), 'r', encoding='utf-8') as f:
        count += 1
        if count % percentil == 0: print(int(count/percentil),'% done')
        texto = f.read()
        texto = texto.split()
        #print(len(texto))
        symbols = [x for x in string.punctuation]
        texto = [p for p in texto if p not in symbols]
        #print(len(texto))
        texto = [p.strip(string.punctuation) for p in texto]
        #print(len(texto))
        texto = [p for p in texto if not p.isdigit()]
        #print(len(texto))
        texto = [p for p in texto if len(p)>1]
        #print(len(texto))
        #print(texto)
        textos.append(texto)
#print(textos)

1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done


### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the synopses. First, I load [NLTK's](http://www.nltk.org/) list of English stop words. [Stop words](http://en.wikipedia.org/wiki/Stop_words) are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [5]:
# load nltk's English stopwords as variable called 'stopwords'
additional_words = ['mr',]
stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [6]:
#Latent Dirichlet Allocation implementation with Gensim
from gensim import corpora, models, similarities 

%time texts = [[word for word in text if word not in stopwords] for text in textos]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/configparser.py:81: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  'PID': str(os.getpid()),
Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


CPU times: user 2min 1s, sys: 212 ms, total: 2min 1s
Wall time: 2min 1s


In [7]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

1008


In [8]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [9]:
len(corpus)

66908

In [10]:
%time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=10)

CPU times: user 26min 58s, sys: 17.3 s, total: 27min 15s
Wall time: 26min 55s


In [11]:
lda.print_topics(5, num_words=20)

[(9,
  '0.012*vg + 0.010*telegrama + 0.010*eh + 0.008*chanceler + 0.008*recebido + 0.008*pt + 0.007*brasil + 0.005*serie + 0.005*pais + 0.005*parte + 0.005*hoje + 0.005*brasemb + 0.004*nr + 0.004*exteriores + 0.004*secreto + 0.004*confidencial + 0.003*política + 0.003*ordem + 0.003*presidente + 0.003*ontem'),
 (5,
  '0.017*united + 0.015*committee + 0.014*general + 0.014*nations + 0.010*resolution + 0.010*assembly + 0.010*international + 0.008*report + 0.007*draft + 0.007*republic + 0.007*states + 0.006*article + 0.006*session + 0.006*shall + 0.006*paragraph + 0.005*meeting + 0.005*council + 0.005*secretary-general + 0.005*commission + 0.004*conference'),
 (2,
  '0.013*países + 0.012*brasil + 0.007*política + 0.006*relações + 0.005*desenvolvimento + 0.005*ser + 0.004*país + 0.004*governo + 0.004*internacional + 0.003*ainda + 0.003*milhões + 0.003*mundo + 0.003*maior + 0.003*cooperação + 0.003*produtos + 0.003*unidos + 0.003*us + 0.003*anos + 0.003*econômica + 0.003*dois'),
 (4,
  '0.01

In [12]:
lda.show_topics()

[(0,
  '0.014*telegrama + 0.010*minuta + 0.008*índice + 0.007*expedido + 0.007*silveira + 0.007*co + 0.007*mm + 0.006*distribuição + 0.005*er + 0.004*via'),
 (1,
  '0.008*countries + 0.008*states + 0.006*would + 0.006*united + 0.006*world + 0.005*brazil + 0.005*international + 0.005*economic + 0.005*new + 0.004*nuclear'),
 (2,
  '0.013*países + 0.012*brasil + 0.007*política + 0.006*relações + 0.005*desenvolvimento + 0.005*ser + 0.004*país + 0.004*governo + 0.004*internacional + 0.003*ainda'),
 (3,
  '0.005*gobierno + 0.004*si + 0.004*argentina + 0.004*paraguay + 0.003*relaciones + 0.003*corpus + 0.003*acuerdo + 0.003*países + 0.003*general + 0.003*brasil'),
 (4,
  '0.015*governo + 0.011*brasil + 0.008*brasileiro + 0.008*argentina + 0.008*vossa + 0.007*itaipu + 0.007*rio + 0.006*ser + 0.006*excelência + 0.005*assunto'),
 (5,
  '0.017*united + 0.015*committee + 0.014*general + 0.014*nations + 0.010*resolution + 0.010*assembly + 0.010*international + 0.008*report + 0.007*draft + 0.007*rep

In [13]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [14]:
topics_matrix[3]

(3,
 [('gobierno', 0.0051305272358267567),
  ('si', 0.0042974492359934158),
  ('argentina', 0.0035305712408054283),
  ('paraguay', 0.0035011880159641351),
  ('relaciones', 0.0034017921828551258),
  ('corpus', 0.003260900784348405),
  ('acuerdo', 0.003245483059608827),
  ('países', 0.0027510791149767625),
  ('general', 0.0026112203400631461),
  ('brasil', 0.0025914041736185907),
  ('pays', 0.0025832008103059559),
  ('artículo', 0.0024703575554266663),
  ('der', 0.0022882724411620744),
  ('plus', 0.0022790277470778543),
  ('cette', 0.0022612768339545109),
  ('reunion', 0.0021316853020462594),
  ('buenos', 0.0021062910472601473),
  ('desarrollo', 0.002070753220597547),
  ('deux', 0.0020099425640887512),
  ('monsieur', 0.0019497843329629192)])

In [15]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      18.136570        1       1  0.190089  0.152938
7      15.885817        1       2  0.188609  0.131669
4      12.392071        1       3  0.195921  0.061283
9      11.612708        1       4  0.141121 -0.036018
8      10.493938        1       5  0.091907 -0.056003
1       9.101342        1       6 -0.385295  0.110124
5       6.500797        1       7 -0.388785  0.127672
6       6.407633        1       8  0.098479 -0.017924
0       5.013909        1       9 -0.089775 -0.381541
3       4.455214        1      10 -0.042270 -0.092201, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
66251  Default  19809.000000           senhor  19809.000000  30.0000  30.0000
44254  Default  21177.000000         ministro  21177.000000  29.0000  29.0000
80447  Default  21468.000000       presidente  21468.000000  28.0000  28.0000
35633  Default  13267.000000        telegrama  13267.000000  27.0000  27.0000
84135  Default  11377.000000           united  11377.000000  26.0000  26.0000
71370  Default  24102.000000         relações  24102.000000  25.0000  25.0000
46391  Default  12454.000000       embaixador  12454.000000  24.0000  24.0000
36263  Default   9641.000000               vg   9641.000000  23.0000  23.0000
25780  Default  29939.000000           países  29939.000000  22.0000  22.0000
27848  Default  13103.000000       excelência  13103.000000  21.0000  21.0000
93467  Default   6832.000000        committee   6832.000000  20.0000  20.0000
48242  Default  12562.000000        república  12562.000000  19.0000  19.0000
77476  Default  23843.000000          governo  23843.000000  18.0000  18.0000
45590  Default   7812.000000          nations   7812.000000  17.0000  17.0000
4298   Default  16902.000000       exteriores  16902.000000  16.0000  16.0000
84797  Default  11339.000000            vossa  11339.000000  15.0000  15.0000
55608  Default   9297.000000          general   9297.000000  14.0000  14.0000
63117  Default   8240.000000               eh   8240.000000  13.0000  13.0000
25279  Default  50439.000000           brasil  50439.000000  12.0000  12.0000
95551  Default   9084.000000           visita   9084.000000  11.0000  11.0000
22044  Default   7892.000000    international   7892.000000  10.0000  10.0000
61175  Default   7954.000000           states   7954.000000   9.0000   9.0000
6332   Default   8533.000000        chanceler   8533.000000   8.0000   8.0000
57877  Default   4655.000000       resolution   4655.000000   7.0000   7.0000
28629  Default   6573.000000        countries   6573.000000   6.0000   6.0000
71494  Default   4624.000000         assembly   4624.000000   5.0000   5.0000
25211  Default  11593.000000  desenvolvimento  11593.000000   4.0000   4.0000
45143  Default   7257.000000         recebido   7257.000000   3.0000   3.0000
51828  Default   6614.000000               pt   6614.000000   2.0000   2.0000
95713  Default   9078.000000        argentina   9078.000000   1.0000   1.0000
...        ...           ...              ...           ...      ...      ...
99759  Topic10    508.883534          derecho    509.773368   3.1093  -6.4114
22521  Topic10    495.810314            senor    496.700161   3.1093  -6.4374
44614  Topic10    486.746532            d'une    487.636348   3.1093  -6.4559
76823  Topic10    483.955532            etats    484.845344   3.1093  -6.4616
17619  Topic10    453.100885            cours    453.990699   3.1091  -6.5275
82450  Topic10    452.768174             sino    453.658059   3.1091  -6.5283
28168  Topic10    449.214682        gobiernos    450.104501   3.1091  -6.5361
47107  Topic10    433.939160           itaipú    434.829025   3.1090  -6.5707
14255  Topic10    427.895329             ello    428.785177   3.1090  -6.5848
3407   Topic10    425.024628     

In [16]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))